In [1]:
import logging
import json
import os
import re
from pathlib import Path
from collections import Counter

import anndata
import pandas as pd
import numpy as np
import tifffile
from einops import rearrange
from skimage.measure import regionprops

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [10]:
import multiplex_imaging_pipeline.utils as utils
import multiplex_imaging_pipeline.ome as ome
import multiplex_imaging_pipeline.segmentation as seg
import multiplex_imaging_pipeline.multiplex_imaging_pipeline as mip
from multiplex_imaging_pipeline.spatial_features import DEFAULT_GATING_STRATEGY

In [6]:
case = 'HT413C1-Th1k4A1'
runs = [
    '2024-01-11_HT413C1_Th1k4A1',
    '2024-01-10_HT413C1_Th1k4A1'
]
output_dir = f'/diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/{case}'

In [7]:
metadata = pd.read_csv('/diskmnt/Projects/Users/estorrs/sandbox/specimen_tracking.tsv', sep='\t')
metadata = metadata[[True if x in runs else False for x in metadata['omero_run_name']]]
metadata

,run_name,omero_run_name,date,case,specimen,bbox,filepath
6,20240111_Human_mCRC_serial_sectrion_HT413C1_Th...,2024-01-11_HT413C1_Th1k4A1,20240111,HT413C1_Th1k4A1,HT413C1_Th1k4A1_U10,"0,7000,0,8000",/diskmnt/primary/CODEX/HTAN/20240111_Human_mCR...
7,20240111_Human_mCRC_serial_sectrion_HT413C1_Th...,2024-01-11_HT413C1_Th1k4A1,20240111,HT413C1_Th1k4A1,HT413C1_Th1k4A1_U26,"11000,20000,2000,9000",/diskmnt/primary/CODEX/HTAN/20240111_Human_mCR...
8,20240111_Human_mCRC_serial_sectrion_HT413C1_Th...,2024-01-11_HT413C1_Th1k4A1,20240111,HT413C1_Th1k4A1,HT413C1_Th1k4A1_U37,"24000,33000,1000,9000",/diskmnt/primary/CODEX/HTAN/20240111_Human_mCR...
9,20240111_Human_mCRC_serial_sectrion_HT413C1_Th...,2024-01-11_HT413C1_Th1k4A1,20240111,HT413C1_Th1k4A1,HT413C1_Th1k4A1_U42,"36000,45000,1000,9000",/diskmnt/primary/CODEX/HTAN/20240111_Human_mCR...
10,20240110_Human_mCRC_Serial_section_HT413C1_Th1...,2024-01-10_HT413C1_Th1k4A1,20240110,HT413C1_Th1k4A1,HT413C1_Th1k4A1_U3,"0,9000,1000,8000",/diskmnt/primary/CODEX/HTAN/20240110_Human_mCR...
11,20240110_Human_mCRC_Serial_section_HT413C1_Th1...,2024-01-10_HT413C1_Th1k4A1,20240110,HT413C1_Th1k4A1,HT413C1_Th1k4A1_U20,"12000,22000,0,8000",/diskmnt/primary/CODEX/HTAN/20240110_Human_mCR...
12,20240110_Human_mCRC_Serial_section_HT413C1_Th1...,2024-01-10_HT413C1_Th1k4A1,20240110,HT413C1_Th1k4A1,HT413C1_Th1k4A1_U32,"24000,34000,0,8000",/diskmnt/primary/CODEX/HTAN/20240110_Human_mCR...
13,20240110_Human_mCRC_Serial_section_HT413C1_Th1...,2024-01-10_HT413C1_Th1k4A1,20240110,HT413C1_Th1k4A1,HT413C1_Th1k4A1_U41,"37000,46000,1000,9000",/diskmnt/primary/CODEX/HTAN/20240110_Human_mCR...


In [14]:
mapping = utils.R_CHANNEL_MAPPING
for fp in metadata['filepath']:
    channels = utils.get_ome_tiff_channels(fp)
    for c in channels:
        assert c in mapping, f'channel {c} is not in channel mapping, to ensure all channel names match consider adding {c} to channel mapping'
        
    print(fp, [mapping[c] for c in channels])

/diskmnt/primary/CODEX/HTAN/20240111_Human_mCRC_serial_sectrion_HT413C1_Th1k4A1_Slide_8/HT413C1_Th1k4A1_U10__20240111.ome.tiff ['DAPI', 'Podoplanin', 'Pan-Cytokeratin', 'HLA-DR', 'CD8', 'SMA', 'CD45', 'Ki67', 'Hep-Par-1', 'GLUT1', 'CK14', 'CD31', 'CCL2', 'E-cadherin', 'MUC2', 'CD11b', 'SOX9', 'CD20', 'FOXP3', 'P21', 'CK7', 'CD68', 'P16', 'CD4', 'PAI1', 'CD3e', 'CK8/18']
/diskmnt/primary/CODEX/HTAN/20240111_Human_mCRC_serial_sectrion_HT413C1_Th1k4A1_Slide_8/HT413C1_Th1k4A1_U26__20240111.ome.tiff ['DAPI', 'Podoplanin', 'Pan-Cytokeratin', 'HLA-DR', 'CD8', 'SMA', 'CD45', 'Ki67', 'Hep-Par-1', 'GLUT1', 'CK14', 'CD31', 'CCL2', 'E-cadherin', 'MUC2', 'CD11b', 'SOX9', 'CD20', 'FOXP3', 'P21', 'CK7', 'CD68', 'P16', 'CD4', 'PAI1', 'CD3e', 'CK8/18']
/diskmnt/primary/CODEX/HTAN/20240111_Human_mCRC_serial_sectrion_HT413C1_Th1k4A1_Slide_8/HT413C1_Th1k4A1_U37__20240111.ome.tiff ['DAPI', 'Podoplanin', 'Pan-Cytokeratin', 'HLA-DR', 'CD8', 'SMA', 'CD45', 'Ki67', 'Hep-Par-1', 'GLUT1', 'CK14', 'CD31', 'CCL2',

In [17]:
nuclei_channels = ['DAPI']
membrane_channels = [
    'Pan-Cytokeratin', 'HLA-DR', 'SMA', 'CD45', 'Hep-Par-1', 'CD31', 'E-cadherin', 'CD68', 'CD3e']
seg_dir = os.path.join(output_dir, 'segmentation')
Path(seg_dir).mkdir(parents=True, exist_ok=True)
for specimen, fp in zip(metadata['specimen'], metadata['filepath']):
    print(specimen)
    prefix = os.path.join(seg_dir, specimen)
    mip.segment_ome(fp, prefix, 25000, nuclei_channels, membrane_channels)

2024-01-30 14:27:05,917 - starting segmentation for /diskmnt/primary/CODEX/HTAN/20240111_Human_mCRC_serial_sectrion_HT413C1_Th1k4A1_Slide_8/HT413C1_Th1k4A1_U10__20240111.ome.tiff
2024-01-30 14:27:05,968 - Checking for cached data


HT413C1_Th1k4A1_U10


2024-01-30 14:27:07,278 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-01-30 14:27:07,279 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-01-30 14:27:07,280 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-01-30 14:27:10,052 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models
2024-01-30 14:27:10.304000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/lib:/usr/lib64:/usr/local/lib:/usr/lib64:
2024-01-30 14:27:10.304089: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-01-30

2024-01-30 14:27:19,113 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-01-30 14:36:16,214 - finished segmentation
2024-01-30 14:36:16,215 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT413C1-Th1k4A1/segmentation/HT413C1_Th1k4A1_U10_nuclei_segmentation.tif
2024-01-30 14:36:16,691 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT413C1-Th1k4A1/segmentation/HT413C1_Th1k4A1_U10_cell_segmentation.tif
2024-01-30 14:36:17,149 - starting segmentation for /diskmnt/primary/CODEX/HTAN/20240111_Human_mCRC_serial_

HT413C1_Th1k4A1_U26


...

2024-01-30 15:28:19,332 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-01-30 15:28:19,333 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-01-30 15:28:19,334 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-01-30 15:28:22,146 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models


2024-01-30 15:28:30,679 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-01-30 15:39:58,056 - finished segmentation
2024-01-30 15:39:58,060 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT413C1-Th1k4A1/segmentation/HT413C1_Th1k4A1_U32_nuclei_segmentation.tif
2024-01-30 15:39:58,657 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT413C1-Th1k4A1/segmentation/HT413C1_Th1k4A1_U32_cell_segmentation.tif
2024-01-30 15:39:59,196 - starting segmentation for /diskmnt/primary/CODEX/HTAN/20240110_Human_mCRC_Serial_

HT413C1_Th1k4A1_U41


2024-01-30 15:39:59,441 - Checking MultiplexSegmentation-9.tar.gz against provided file_hash...
2024-01-30 15:39:59,441 - MultiplexSegmentation-9.tar.gz with hash a1dfbce2594f927b9112f23a0a1739e0 already available.
2024-01-30 15:39:59,442 - Extracting /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz
2024-01-30 15:40:02,361 - Successfully extracted /home/estorrs/.deepcell/models/MultiplexSegmentation-9.tar.gz into /home/estorrs/.deepcell/models


2024-01-30 15:40:11,346 - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
/diskmnt/Projects/Users/estorrs/miniconda3/envs/imaging_analysis_v2/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '
2024-01-30 15:50:09,885 - finished segmentation
2024-01-30 15:50:09,886 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT413C1-Th1k4A1/segmentation/HT413C1_Th1k4A1_U41_nuclei_segmentation.tif
2024-01-30 15:50:10,268 - writing /diskmnt/Projects/Users/estorrs/imaging-analysis/results/serial_sections/HT413C1-Th1k4A1/segmentation/HT413C1_Th1k4A1_U41_cell_segmentation.tif
